<a href="https://colab.research.google.com/github/robitussin/CCADMACL_EXERCISES/blob/main/Exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 2: Use Gradient Boost for Regression

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
Submit your results to:
https://www.kaggle.com/competitions/playground-series-s4e12/overview



In [97]:
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import datasets, ensemble
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from hyperopt import fmin, tpe, hp, Trials
import lightgbm as lgb


## Dataset
Train, test and sample submission file can be found in this link
https://www.kaggle.com/competitions/playground-series-s4e12/data

## 1. Load the Data

In [98]:
test_path  = "test.csv"
train_path = "train.csv"

In [99]:
# put your answer here
df_train = pd.read_csv(train_path, na_values=['NA', '', '?'])
df_test = pd.read_csv(test_path, na_values=['NA', '', '?'])

## 2. Perform Data preprocessing

In [100]:
test_ids = df_train['id']
target_column = 'Premium Amount'

categorical_columns = df_train.select_dtypes(include=['object']).columns
numerical_columns = df_train.select_dtypes(exclude=['object']).columns

print("Target Column:", target_column)
print("\nCategorical Columns:", categorical_columns.tolist())
print("\nNumerical Columns:", numerical_columns.tolist())

Target Column: Premium Amount

Categorical Columns: ['Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location', 'Policy Type', 'Policy Start Date', 'Customer Feedback', 'Smoking Status', 'Exercise Frequency', 'Property Type']

Numerical Columns: ['id', 'Age', 'Annual Income', 'Number of Dependents', 'Health Score', 'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration', 'Premium Amount']


In [101]:
df_train

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,...,NaN,5.0,372.0,3.0,2023-05-03 15:21:39.257696,Poor,No,Daily,Apartment,1303.0
1199996,1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,...,NaN,10.0,597.0,4.0,2022-09-10 15:21:39.134960,Poor,No,Weekly,Apartment,821.0
1199997,1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,...,0.0,19.0,NaN,6.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo,371.0
1199998,1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,...,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment,596.0


In [102]:
# Describe
df_train.describe().round(3)

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,1200000.000,1181295.000,1155051.000,1090328.000,1125924.000,835971.000,1199994.000,1062118.000,1199999.000,1200000.000
mean,599999.500,41.146,32745.218,2.010,25.614,1.003,9.570,592.924,5.018,1102.545
std,346410.306,13.540,32179.506,1.417,12.203,0.983,5.776,149.982,2.594,864.999
min,0.000,18.000,1.000,0.000,2.012,0.000,0.000,300.000,1.000,20.000
25%,299999.750,30.000,8001.000,1.000,15.919,0.000,5.000,468.000,3.000,514.000
50%,599999.500,41.000,23911.000,2.000,24.579,1.000,10.000,595.000,5.000,872.000
75%,899999.250,53.000,44634.000,3.000,34.527,2.000,15.000,721.000,7.000,1509.000
max,1199999.000,64.000,149997.000,4.000,58.976,9.000,19.000,849.000,9.000,4999.000


## 3. Create a Pipeline

#### Mutate features

In [115]:
# Define features and target
numerical_features = [
     'Age', 'Annual Income', 'Number of Dependents', 'Health Score', 
    'Vehicle Age', 'Credit Score', 'Insurance Duration'
]
categorical_features = [
    'Gender', 'Marital Status', 'Education Level', 'Occupation', 'Location', 'Policy Type', 'Policy Start Date',
    'Smoking Status', 'Exercise Frequency', 'Property Type'
]

target_column = 'Premium Amount'

#### Split dependent and independent data

In [116]:
# Split train data into features and target
X = df_train.drop(columns=[target_column, 'id', 'Previous Claims', 'Customer Feedback'])
y = df_train[target_column]

In [117]:
print(X.columns)
print(df_test.columns)

Index(['Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Vehicle Age', 'Credit Score',
       'Insurance Duration', 'Policy Start Date', 'Smoking Status',
       'Exercise Frequency', 'Property Type'],
      dtype='object')
Index(['id', 'Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Policy Start Date',
       'Customer Feedback', 'Smoking Status', 'Exercise Frequency',
       'Property Type'],
      dtype='object')


#### Handle missing values 

In [118]:
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

In [119]:
# Preprocess train and test data
X_processed = preprocessor.fit_transform(X)
test_processed = preprocessor.transform(df_test.drop(columns=['id', 'Previous Claims', 'Customer Feedback']))

In [120]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=42)

## 4. Train the Model

In [80]:
import numpy as np
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, Trials

# Objective function to optimize
def objective(params):
    # Cast min_data_in_leaf to an integer
    params['min_data_in_leaf'] = int(params['min_data_in_leaf'])
    
    # Create and train the model with the given hyperparameters
    model = lgb.train({**params, 'seed': 42}, lgb.Dataset(X_processed, label=y))

    # Evaluate the model on validation data
    y_pred = model.predict(X_val)
    mse = np.mean((y_pred - y_val) ** 2)  # Use MSE for evaluation

    return mse  # Minimize MSE

# Define the hyperparameter space
space = {
    'boosting_type': hp.choice('boosting_type', ['dart', 'gbdt', 'goss', 'rf']),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
    'feature_fraction': hp.uniform('feature_fraction', 0.7, 1.0),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1.0),
    
    # Add more hyperparameters specific to LightGBM
    'num_leaves': hp.choice('num_leaves', [31, 63, 127, 255]),  # Number of leaves in each tree
    'max_depth': hp.choice('max_depth', [-1, 5, 10, 15]),  # Maximum depth of each tree (-1 means no limit)
    'min_data_in_leaf': hp.quniform('min_data_in_leaf', 20, 100, 10),  # Minimum number of data points in each leaf
    'lambda_l1': hp.uniform('lambda_l1', 0, 10),  # L1 regularization
    'lambda_l2': hp.uniform('lambda_l2', 0, 10),  # L2 regularization
    'min_gain_to_split': hp.uniform('min_gain_to_split', 0, 0.2),  # Minimum gain to make a further partition
    'metric': hp.choice('metric', ['l2', 'l1', 'rmse'])  # Metric to evaluate (mean squared error, etc.)
}


In [82]:
# Run the optimization
trials = Trials()
best_hyperparams = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
                                                       

[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 100, number of used features: 0
[LightGBM] [Info] Start training from score 0.531851   
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requ

In [84]:
best_hyperparams

{'bagging_fraction': 0.7956468068680306,
 'boosting_type': 1,
 'feature_fraction': 0.7379965154821778,
 'lambda_l1': 0.04785130938516419,
 'lambda_l2': 0.011134187852256575,
 'learning_rate': 0.03458876222385471,
 'max_depth': 2,
 'metric': 0,
 'min_data_in_leaf': 40.0,
 'min_gain_to_split': 0.07667223442985392,
 'num_leaves': 1}

In [121]:
best_hyperparams_copied = {'bagging_fraction': 0.7956468068680306,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.7379965154821778,
 'lambda_l1': 0.04785130938516419,
 'lambda_l2': 0.011134187852256575,
 'learning_rate': 0.03458876222385471,
 'max_depth': 14,
 'metric': 0,
 'min_data_in_leaf': 40,
 'min_gain_to_split': 0.07667223442985392,
 'num_leaves': 300}

final_model = lgb.train({**best_hyperparams_copied, 'seed': 42}, lgb.Dataset(X_processed, label=y))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.087051 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10501
[LightGBM] [Info] Number of data points in the train set: 1200000, number of used features: 4832
[LightGBM] [Info] Start training from score 1102.544822


## 5. Evaluate the Model

In [122]:
y_pred = final_model.predict(X_processed)

rmsle = np.sqrt(mean_squared_log_error(y, y_pred))  
rmse = np.sqrt(mean_squared_error(y, y_pred))

# Display performance metrics
print(f"\nPerformance Metrics:\n{'-'*30}")
print(f"RMSLE: {rmsle:.4f}")
print(f"RMSE: {rmse:.4f}")


Performance Metrics:
------------------------------
RMSLE: 1.1536
RMSE: 849.3969


## Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [123]:
# Make predictions on the test set
test_predictions = final_model.predict(test_processed, num_iteration=final_model.best_iteration)
submission = pd.DataFrame({'id': df_test['id'], 'Premium Amount': test_predictions})
submission.to_csv("submission.csv", index=False)